<a href="https://colab.research.google.com/github/theoem/Data-Linkage/blob/master/Data-Linkage/Record-Linking-Fuzzy-Matching-Teodoras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC8381 Record Linking - Teodoras Celencevicius 190364247


Firstly, we will need to load the needed libraries

In [879]:
!pip install fuzzymatcher

In [880]:
!pip install recordlinkage

In [881]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage
from sklearn.metrics import precision_recall_fscore_support

## Fuzzymatcher approach

Loading the data

In [882]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv'
)

In [883]:
acm_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [884]:
dblp_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


Deleting any rows where author is not defined

In [885]:
acm_data.dropna(subset=['authors'], inplace=True)
dblp_data.dropna(subset=['authors'], inplace=True)

### Comparing the data sets

In [886]:
## dataframe sizes

print("raw dataframe number of records :")
print("acm_data:  {x}".format(x=acm_data['id'].count()))
print("dblp_data:  {x}".format(x=dblp_data['id'].count()))

raw dataframe number of records :
acm_data:  2280
dblp_data:  2616


In [887]:
cond = acm_data['authors'].str.startswith('A')
acm_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

,authors,title
1054,"A. A. Diwan, Sanjeeva Rane, S. Seshadri, S. Su...",Clustering Techniques for Minimizing External ...
1741,"A. Biliris, S. Dar, N. Gehani, H. V. Jagadish,...",ASSET: a system for supporting extended transa...
235,"A. Biliris, T. A. Funkhouser, W. O'Connell, E....",BeSS: storage support for interactive visualiz...
1565,"A. Dogac, C. Dengi, E. Kilic, G. Ozhan, F. Ozc...",METU interoperable database system
218,"A. Formica, H. D. Groger, M. Missikoff",An efficient method for checking object-orient...
1632,"A. Kemper, G. Moerkotte, K. Peithner, M. Stein...",Optimizing disjunctive queries with expensive ...
118,"A. M. Ouksel, A. Sheth",Semantic interoperability in global informatio...
2186,"A. Maedche, B. Motik, L. Stojanovic",Managing multiple and distributed ontologies o...
1529,"A. P. Buchmann, A. Deutsch, J. Zimmermann, M. ...",The REACH active OODBMS
1439,"A. Prasad Sistla, Clement T. Yu, Chengwen Liu,...",Similarity based Retrieval of Pictures Using I...


In [888]:
cond = dblp_data['authors'].str.startswith('A')
dblp_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

,authors,title
207,"A. A. Diwan, S. Sudarshan, Sanjeeva Rane, S. S...",Clustering Techniques for Minimizing External ...
1279,"A. Kumaran, Jayant R. Haritsa",On the Costs of Multilingualism in Database Sy...
433,"A. Prasad Sistla, Isabel F. Cruz, Ashfaq A. Kh...",Research Activities in Database Management and...
1587,"A. Prasad Sistla, Ouri Wolfson, Yixiu Huang",Data Replication for Mobile Computers
1128,"A. Prasad Sistla, Robert H. Sloan, Yelena Yesh...",Towards a Theory of Cost Management for Digita...
1642,A. R. Dasgupta,Large Databases for Remote Sensing and GIS
703,"A. R. Hurson, Simin H. Pakzad, M. W. Bright",Automated Resolution of Semantic Heterogeneity...
2057,"Aashu Virmani, Tomasz Imielinski",DataMine - Interactive Rule Discovery System
396,"Abdelsalam Helal, Hua Li",Rainbow: Distributed Database System for Class...
2033,"Abraham Silberschatz, Christos Faloutsos, Yoss...",Modeling Skewed Distribution Using Multifracta...


Right of the bat, there are some simmilarities that can be seen, but it's appearent that a simple join won't cut it.

In [889]:
acm_data[acm_data['authors'].str.match('A. Prasad')][['authors', 'title']]

,authors,title
134,"A. Prasad Sistla, Ouri Wolfson, Yelena Yesha, ...",Towards a theory of cost management for digita...
1439,"A. Prasad Sistla, Clement T. Yu, Chengwen Liu,...",Similarity based Retrieval of Pictures Using I...
1511,"A. Prasad Sistla, Ouri Wolfson",Temporal conditions and integrity constraints ...
2221,"A. Prasad Sistla, Clement T. Yu, R. Haddad",Reasoning About Spatial Relationships in Pictu...


In [890]:
dblp_data[dblp_data['authors'].str.match('A. Prasad')][['authors', 'title']]

,authors,title
433,"A. Prasad Sistla, Isabel F. Cruz, Ashfaq A. Kh...",Research Activities in Database Management and...
1128,"A. Prasad Sistla, Robert H. Sloan, Yelena Yesh...",Towards a Theory of Cost Management for Digita...
1587,"A. Prasad Sistla, Ouri Wolfson, Yixiu Huang",Data Replication for Mobile Computers


This can be seen in the example above where the entry for the title `Towards a theory of cost management for digita...` can be seen in both data sets, but the strings doesn't match for both authors and title.

### Implementing fuzzymatching

In [891]:
acm_data.columns

Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')

In [892]:
dblp_data.columns

Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')

In [893]:
# Columns to match on from acm_data
left_on = ['title', 'authors', 'venue', 'year']

# Columns to match on from dblp_data
right_on = ['title', 'authors', 'venue', 'year']

Linking the data

In [894]:
linked = fuzzymatcher.link_table(acm_data,
                                 dblp_data,
                                 left_on,
                                 right_on,
                                 left_id_col='id',
                                 right_id_col='id')

In [895]:
linked.head()

,__id_left,__id_right,match_score,match_rank,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
0,304586,conf/sigmod/VossenW99,2.559548,1,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen",International Conference on Management of Data,SIGMOD Conference,1999,1999
1,304587,conf/sigmod/CruzJ99,2.721471,1,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz",International Conference on Management of Data,SIGMOD Conference,1999,1999
2,304589,conf/sigmod/BouguettayaBH99,4.568259,1,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR...","Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan...",International Conference on Management of Data,SIGMOD Conference,1999,1999
3,304590,conf/sigmod/BaruGLMPVC99,3.630595,1,XML-based information mediation with MIX,XML-Based Information Mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha...",International Conference on Management of Data,SIGMOD Conference,1999,1999
4,304582,conf/sigmod/BrodskySCE99,2.969856,1,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict...",International Conference on Management of Data,SIGMOD Conference,1999,1999


Now perform the actual matches

In [896]:
matched_results = fuzzymatcher.fuzzy_left_join(acm_data,
                                               dblp_data,
                                               left_on,
                                               right_on,
                                               left_id_col='id',
                                               right_id_col='id')

Looking into some of the records, it can be seen that alteast some data was linked correctly

In [897]:
matched_results[["authors_left", "authors_right"]].head()

,authors_left,authors_right
0,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen"
1,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz"
2,"Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan..."
3,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha..."
4,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict..."


### Investigating the linkage success further

Reorder the columns to make viewing easier

In [898]:
cols = [
    "best_match_score", "title_left",	"title_right", "authors_left", 
    "authors_right", "venue_left",	"venue_right", "year_left", "year_right",
    "id_left", "id_right"
]

Let's see the best matches

In [899]:
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
25258,9.284321,The Propel Distributed Services Platform,The Propel Distributed Services Platform,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",Very Large Data Bases,VLDB,2001,2001,672199,conf/vldb/CareyKRLABFLOPQRSSUVWY01
34608,9.233475,A mapping mechanism to support bitmap index an...,A mapping mechanism to support bitmap index an...,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",ACM SIGMOD Record,SIGMOD Record,2003,2003,777000,journals/sigmod/ChongFTSYKDJJ03
29304,9.165640,TSQL2 language specification,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang Käfer, Curtis E. Dyreson, Arie Segev,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,181562,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94
28640,9.091846,A TSQL2 tutorial,A TSQL2 Tutorial,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,187449,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94a
427,8.605697,PowerBookmarks: a system for personalizable We...,PowerBookmarks: A System for Personalizable We...,"Wen-Syan Li, Quoc Vu, Edward Chang, Divyakant ...","Kazuyuki Shimazu, Sougata Mukherjea, Yi-Leh Wu...",International Conference on Management of Data,SIGMOD Conference,1999,1999,304578,conf/sigmod/LiVCAHMWBCHIKSS99


Now the worst matches

In [900]:
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
19682,-1.972738,David DeWitt speaks out: on rethinking the CS ...,Gio Wiederhold Speaks Out on Moving into Acade...,Marianne Winslett,Marianne Winslett,ACM SIGMOD Record,SIGMOD Record,2002,2001,565127,journals/sigmod/Winslett01a
29418,-1.682680,A consensus glossary of temporal database conc...,A TSQL2 Tutorial,"Curtis Dyreson, Fabio Grandi, Wolfgang K&#228;...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,181560,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94a
5696,-1.278196,Challenges in automating declarative business ...,Expressing Business Rules,Val Huber,Ronald G. Ross,International Conference on Management of Data,SIGMOD Conference,2000,2000,335455,conf/sigmod/Ross00
34079,-1.271728,Abstracts of invited industrial track presenta...,Objectivity Industrial Exhibit,Zachary Ives,Objectivity,International Conference on Management of Data,VLDB,2003,1998,872833,conf/vldb/Objectivity98
5487,-1.263943,Going beyond personalization: rule engines at ...,Delivering High Availability for Inktomi Searc...,Eric Kintzer,Eric A. Brewer,International Conference on Management of Data,SIGMOD Conference,2000,1998,335459,conf/sigmod/Brewer98


Some of these matches are obviously incorrect. Which indicates that the matching was not perfect. This means that a threshold has to be found from which most of the data linked is correct in `matched_results`.

### Finding the threshold of the `best_match_score` from which the data is linked correctly.

Look at the matches around 1

In [901]:
matched_results[cols].query("best_match_score <= 1").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
3503,0.995988,MQseries and CICS link for Lotus Notes,MQSeries and CICS Link for Lotus Notes,Lotus Development Corp.,?,ACM SIGMOD Record,SIGMOD Record,1996,1996,381900,journals/sigmod/Anonymous96
2297,0.995680,Keynote Address,Keynote Address,Robert S. Epstein,Robert S. Epstein,International Conference on Management of Data,SIGMOD Conference,1995,1995,277955,conf/sigmod/Epstein95
25428,0.990004,The Oracle Warehouse,The Oracle Warehouse,Gary Hallmark,Gary Hallmark,Very Large Data Bases,VLDB,1995,1995,673156,conf/vldb/Hallmark95
25266,0.984455,Altruistic locking,Altruistic Locking,"Kenneth Salem, H&#233;ctor Garc&#237;a-Molina,...","Jeannie Shands, Kenneth Salem, Hector Garcia-M...",ACM Transactions on Database Systems (TODS),ACM Trans. Database Syst.,1994,1994,174639,journals/tods/SalemGS94
18422,0.973914,Mid-tier caching: the TimesTen approach,Mid-tier caching: the TimesTen approach,The TimesTen Team,Times-Ten Team,International Conference on Management of Data,SIGMOD Conference,2002,2002,564761,conf/sigmod/Team02


Look at the matches around 0.5

In [902]:
matched_results[cols].query("best_match_score <= 0.50").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
1723,0.488228,Semantic interoperability in global informatio...,Semantic Interoperability in Global Informatio...,"A. M. Ouksel, A. Sheth","Amit P. Sheth, Aris M. Ouksel",ACM SIGMOD Record,SIGMOD Record,1999,1999,309849,journals/sigmod/OukselS99
35210,0.484983,Monotonic complements for independent data war...,Monotonic complements for independent data war...,"D. Laurent, J. Lechtenb&#246;rger, N. Spyratos...","Gottfried Vossen, Jens Lechtenbörger, Dominiqu...",The VLDB Journal &mdash; The International Jou...,VLDB J.,2001,2001,767152,journals/vldb/LaurentLSV01
24854,0.480499,Functional-join processing,Functional-Join Processing,"R. Braumandl, J. Claussen, A. Kemper, D. Kossmann","Reinhard Braumandl, Alfons Kemper, Donald Koss...",The VLDB Journal &mdash; The International Jou...,VLDB J.,2000,2000,764214,journals/vldb/BraumandlCKK00
11087,0.476856,Reminiscences on influential papers,Reminiscences on Influential Papers,Kenneth A. Ross,"James R. Hamilton, Limsoon Wong, Kenneth A. Ro...",ACM SIGMOD Record,SIGMOD Record,2001,2001,603882,journals/sigmod/RossHLW01
11287,0.476856,Reminiscences on Influential Papers,Reminiscences on Influential Papers,Kenneth A. Ross,"James R. Hamilton, Limsoon Wong, Kenneth A. Ro...",ACM SIGMOD Record,SIGMOD Record,2001,2001,604262,journals/sigmod/RossHLW01


Hitting the point when the matching is not that obvious and hints for error in data. For example all of the columms investigates are talking about the same thing, but the year is very different.

In [903]:
matched_results[cols].query("best_match_score <= 0.0").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
3512,-0.042062,An Illustra technical white paper,Illustra's Web DataBlade Module,John Gaffney,John Gaffney,ACM SIGMOD Record,SIGMOD Record,1996,1996,381899,journals/sigmod/Gaffney96
16576,-0.061370,What will be,What Will Be - Book Review,Michael Dertouzos,Paul W. P. J. Grefen,ACM SIGMOD Record,SIGMOD Record,2002,2002,507356,journals/sigmod/Grefen02
14183,-0.067738,Research centers,"Workflow Automation: Applications, Technology,...",Amit Sheth,Amit P. Sheth,ACM SIGMOD Record,SIGMOD Conference,2001,1995,604285,conf/sigmod/Sheth95
22695,-0.067738,Research centers,"Workflow Automation: Applications, Technology,...",Amit Sheth,Amit P. Sheth,ACM SIGMOD Record,SIGMOD Conference,2003,1995,641005,conf/sigmod/Sheth95
21205,-0.067738,Research centers,"Workflow Automation: Applications, Technology,...",Amit Sheth,Amit P. Sheth,ACM SIGMOD Record,SIGMOD Conference,2002,1995,637434,conf/sigmod/Sheth95


Looking into matches lower than `0.30`, it's beggining harder to justify some of the matches, therefore the threshold chosen is `0.40`

In [904]:
matched_results[cols].query("best_match_score <= 0.30").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
17293,0.237184,Industry perspective,Industry Perspectives,Len Seligman,Leonard J. Seligman,ACM SIGMOD Record,SIGMOD Record,2002,1997,507361,journals/sigmod/Seligman97
39406,0.236657,In-context peer-to-peer information filtering ...,Mining the World Wide Web: An Information Sear...,Aris M. Ouksel,Aris M. Ouksel,ACM SIGMOD Record,SIGMOD Record,2003,2002,945735,journals/sigmod/Ouksel02
14535,0.207694,Special section on semantic web and data manag...,Amicalola Report: Database and Information Sys...,"Robert Meersman, Amit Sheth","Robert Meersman, Amit P. Sheth",ACM SIGMOD Record,SIGMOD Record,2002,2002,637412,journals/sigmod/ShethM02
24568,0.158733,Introduction,Introduction (Special Issue on Multimedia Data...,"M. Tamer &#214;zsu, Stavros Christodoulakis","M. Tamer Özsu, Stavros Christodoulakis",The VLDB Journal &mdash; The International Jou...,VLDB J.,1998,1998,765537,journals/vldb/OzsuC98
9824,0.117890,SQL/XML is making good progress,SQL Multimedia and Application Packages (SQL/MM),"Andrew Eisenberg, Jim Melton","Jim Melton, Andrew Eisenberg",ACM SIGMOD Record,SIGMOD Record,2002,2001,565141,journals/sigmod/MeltonE01


### Saving the results to csv for future investigation

In [905]:
cols

['best_match_score',
 'title_left',
 'title_right',
 'authors_left',
 'authors_right',
 'venue_left',
 'venue_right',
 'year_left',
 'year_right',
 'id_left',
 'id_right']

In [906]:
matched_results_final = matched_results[cols].query("best_match_score >= 0.40").sort_values(
    by=['id_left'], ascending=True)

Changing the id names and values to match the perfect match dataset.

In [907]:
matched_results_final[['id_left', 'id_right']].rename(
    columns={"id_left": "idACM", "id_right": "idDBLP"}).to_csv(
        'fuzzy_matching.csv', index=False)

In [908]:
matched_results_final.shape

(2221, 11)

## Python Record Linkage Toolkit approach


Loading the data again, with index column specified as it's needed for the record linkage toolkit

In [909]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv',
    index_col='id'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv',
    index_col='id'
)


acm_data.dropna(subset=['authors'], inplace=True)
dblp_data.dropna(subset=['authors'], inplace=True)

acm_data['year'] = acm_data['year'].apply(str)
dblp_data['year'] = dblp_data['year'].apply(str)

### Building the indexer

In [910]:
indexer_full = recordlinkage.Index()
indexer_block = recordlinkage.Index()

indexer_full.full()
indexer_block.block(left_on='title', right_on='title')


<Index>

Let's see how many matches we want to do with each indexer

In [911]:
candidates_block = indexer_block.index(acm_data, dblp_data)
len(candidates_block)

984

In [912]:
candidates_full = indexer_full.index(acm_data, dblp_data)
len(candidates_full)

5964480

In [913]:
print(acm_data.columns)
print(dblp_data.columns)

Index(['Unnamed: 0', 'title', 'authors', 'venue', 'year'], dtype='object')
Index(['Unnamed: 0', 'title', 'authors', 'venue', 'year'], dtype='object')


using Levenshtein comparison method as it works better with long strings. which the title, authors and venue entries are.

[Source](https://srinivas-kulkarni.medium.com/jaro-winkler-vs-levenshtein-distance-2eab21832fd6)

In [914]:
def compare(candidates):
  compare = recordlinkage.Compare()
  compare.exact('year', 'year', label='year')
  compare.string('authors',
                'authors',
                method='levenshtein',
                threshold=0.85,
                label='authors')
  compare.string('title',
                'title',
                method='levenshtein',
                threshold=0.85,
                label='title')
  compare.string('venue',
                'venue',
                method='levenshtein',
                threshold=0.85,
                label='venue')
  return compare.compute(candidates, acm_data,
                            dblp_data)

In [915]:
# Takes 7 minutes using the full index.
# <1s using blocking

features_full = compare(candidates_full)
features_block = compare(candidates_block)

In [916]:
features_full.describe()

,year,authors,title,venue
count,5.964480e+06,5.964480e+06,5.964480e+06,5964480.0
mean,1.000964e-01,2.813322e-04,3.975200e-04,0.0
std,3.001285e-01,1.677060e-02,1.993394e-02,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [917]:
features_block.describe()

,year,authors,title,venue
count,984.000000,984.000000,984.0,984.0
mean,0.930894,0.321138,1.0,0.0
std,0.253763,0.467151,0.0,0.0
min,0.000000,0.000000,1.0,0.0
25%,1.000000,0.000000,1.0,0.0
50%,1.000000,0.000000,1.0,0.0
75%,1.000000,1.000000,1.0,0.0
max,1.000000,1.000000,1.0,0.0


What are the score totals?

In [918]:
features_full.sum(axis=1).value_counts().sort_index(ascending=False)

3.0        669
2.0       1711
1.0     595643
0.0    5366457
dtype: int64

In [919]:
features_block.sum(axis=1).value_counts().sort_index(ascending=False)

3.0    304
2.0    624
1.0     56
dtype: int64

indexer_block has both higher score totals and faster performance. <1s compared to indexer_full almost 7 minutes. Even though, index_full had a lot more match candidates, the mean score decreases greatly. 

For best efficiency indexer_block was used for final results.

In [920]:
features = features_block

### Get the potential matches

In [921]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()

checking if the matching was successful

In [922]:
print(potential_matches.shape)
potential_matches['Score'] = potential_matches.loc[:, 'authors':'authors'].sum(axis=1)
potential_matches.tail()

(928, 6)


,id_1,id_2,year,authors,title,venue,Score
923,672978,conf/vldb/JonscherD94,1,0.0,1.0,0.0,0.0
924,950483,journals/vldb/RamamurthyDS03,1,0.0,1.0,0.0,0.0
925,672977,conf/vldb/KemperK94,1,1.0,1.0,0.0,1.0
926,672980,conf/vldb/Guting94,1,0.0,1.0,0.0,0.0
927,672979,conf/vldb/WienerN94,1,0.0,1.0,0.0,0.0


In [923]:
acm_data.loc[672978,:]

Unnamed: 0                                                 2287
title         An Approach for Building Secure Database Feder...
authors                        Dirk Jonscher, Klaus R. Dittrich
venue                                     Very Large Data Bases
year                                                       1994
Name: 672978, dtype: object

In [924]:
dblp_data.loc['conf/vldb/JonscherD94',:]

Unnamed: 0                                                 1396
title         An Approach for Building Secure Database Feder...
authors                        Klaus R. Dittrich, Dirk Jonscher
venue                                                      VLDB
year                                                       1994
Name: conf/vldb/JonscherD94, dtype: object

### Comparing the data between the matched data sets

Add some convenience columns for comparing data

In [925]:
acm_data['acm_authors_lookup'] = acm_data[[
    'year', 'authors', 'title', 'venue'
]].apply(lambda x: '_'.join(x), axis=1)

In [926]:
dblp_data['dblp_authors_lookup'] = dblp_data[[
     'year', 'authors', 'title', 'venue'
]].apply(lambda x: '_'.join(x), axis=1)

changing the index name to identify it between the two dataframes.

In [927]:
acm_data_lookup = acm_data[['acm_authors_lookup']].reset_index().rename(columns={"id": "id_1"})
dblp_data_lookup = dblp_data[['dblp_authors_lookup']].reset_index().rename(columns={"id": "id_2"})

In [928]:
acm_data_lookup.head()

,id_1,acm_authors_lookup
0,304586,"1999_Gottfried Vossen, Mathias Weske_The WASA2..."
1,304587,"1999_Isabel F. Cruz, Kimberly M. James_A user-..."
2,304589,"1999_Athman Bouguettaya, Boualem Benatallah, L..."
3,304590,"1999_Chaitan Baru, Amarnath Gupta, Bertram Lud..."
4,304582,"1999_Alexander Brodsky, Victor E. Segal, Jia C..."


In [929]:
dblp_data_lookup.head()

,id_2,dblp_authors_lookup
0,journals/sigmod/Mackay99,1999_D. Scott Mackay_Semantic Integration of E...
1,conf/vldb/PoosalaI96,"1996_Viswanath Poosala, Yannis E. Ioannidis_Es..."
2,conf/vldb/PalpanasSCP02,"2002_Themistoklis Palpanas, Richard Sidle, Ham..."
3,conf/vldb/GardarinGT96,"1996_Zhao-Hui Tang, Georges Gardarin, Jean-Rob..."
4,conf/vldb/HoelS95,"1995_Erik G. Hoel, Hanan Samet_Benchmarking Sp..."


Adding the lookup column to the motential matches dataframe for comparison covnenience

For acm_data dataframe

In [930]:
acm_data_merge = potential_matches.merge(acm_data_lookup, how='left')

In [931]:
acm_data_merge.head()

,id_1,id_2,year,authors,title,venue,Score,acm_authors_lookup
0,277955,conf/sigmod/Epstein95,1,1.0,1.0,0.0,1.0,1995_Robert S. Epstein_Keynote Address_Interna...
1,277955,conf/sigmod/Ellison95,1,0.0,1.0,0.0,0.0,1995_Robert S. Epstein_Keynote Address_Interna...
2,276377,conf/sigmod/OlstonWACELSS98,1,0.0,1.0,0.0,0.0,"1998_Chris Olston, Allison Woodruff, Alexander..."
3,276367,conf/sigmod/Aulakh98,1,1.0,1.0,0.0,1.0,1998_Kamar Aulakh_About Quark Digital Media Sy...
4,304211,journals/vldb/MaratheS02,0,1.0,1.0,0.0,1.0,"1999_Arunprasad P. Marathe, Kenneth Salem_Quer..."


And merging it with the dblp_data

In [932]:
final_merge = acm_data_merge.merge(dblp_data_lookup, how='left')

Rearranging the columns for better view

In [933]:
cols = [
    'id_1', 'id_2', 'Score', 'acm_authors_lookup',
    'dblp_authors_lookup'
]

The matching was successful

In [934]:
final_merge[cols].sort_values(by=['id_1', 'Score'], ascending=False)

,id_1,id_2,Score,acm_authors_lookup,dblp_authors_lookup
795,959087,journals/sigmod/Hainaut03,1.0,2003_Jean-Luc Hainaut_Research in database eng...,2003_Jean-Luc Hainaut_Research in database eng...
794,959086,journals/sigmod/KambhampatiK03,1.0,"2003_Subbarao Kambhampati, Craig A. Knoblock_I...","2003_Subbarao Kambhampati, Craig A. Knoblock_I..."
797,959085,journals/sigmod/MotroA03,1.0,"2003_Amihai Motro, Troels Andreasen_Report on ...","2003_Amihai Motro, Troels Andreasen_Report on ..."
796,959084,journals/sigmod/Giorgini03,1.0,2003_Paolo Giorgini_Agent-Oriented software en...,2003_Paolo Giorgini_Agent-Oriented software en...
849,959083,journals/sigmod/LenzVJS03,0.0,"2003_Hans J. Lenz, Panos Vassiliadis, Manfred ...","2003_Manfred A. Jeusfeld, Hans-Joachim Lenz, P..."
...,...,...,...,...,...
745,234900,journals/sigmod/Dogac96,1.0,1996_Asuman Dogac_Middle East Technical Univer...,1996_Asuman Dogac_Middle East Technical Univer...
741,219774,journals/sigmod/Stacey95,1.0,"1995_Doug Stacey_Replication: DB2, Oracle, or ...","1995_Doug Stacey_Replication: DB2, Oracle, or ..."
740,212019,journals/sigmod/RisheSBDOALOSS95,0.0,"1995_Naphtali Rishe, Wei Sun, David Barton, Yi...","1995_Leonard Loureiro, Yi Deng, Carlos Ordonez..."
743,181569,journals/sigmod/Winslett94,1.0,1994_Marianne Winslett_Progress on HPCC and NI...,1994_Marianne Winslett_Progress on HPCC and NI...


### Saving the results to csv for future investigation

Changing the id names and values to match the perfect match dataset.

In [935]:
final_merge[['id_1', 'id_2']].rename(columns={"id_1": "idACM", "id_2": "idDBLP"}).to_csv('record_linkage.csv',
                                                  index=False)

## Reporting the quality of the matches


### Loading the data

Loading the provided perfect dataset and fuzzy and record datasets that were produced earlier from fuzzymatcher and python record linkage toolkits accordingly.

In [967]:
perfect = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv'
)
fuzzy = pd.read_csv('fuzzy_matching.csv')
record = pd.read_csv('record_linkage.csv')

In [968]:
perfect = perfect[['idACM', 'idDBLP']].sort_values(by=['idACM'])

In [969]:
perfect

,idACM,idDBLP
1334,174639,journals/tods/SalemGS94
1331,174640,journals/tods/OliverS94
1332,174641,journals/tods/TendickM94
1333,174642,journals/tods/CliffordC94
1326,176568,journals/tods/RosenthalR94
...,...,...
796,959083,journals/sigmod/LenzVJS03
797,959084,journals/sigmod/Giorgini03
798,959085,journals/sigmod/MotroA03
799,959086,journals/sigmod/KambhampatiK03


In [970]:
fuzzy

,idACM,idDBLP
0,174639,journals/tods/SalemGS94
1,174640,journals/tods/OliverS94
2,174641,journals/tods/TendickM94
3,174642,journals/tods/CliffordC94
4,176568,journals/tods/RosenthalR94
...,...,...
2216,959083,journals/sigmod/LenzVJS03
2217,959084,journals/sigmod/Giorgini03
2218,959085,journals/sigmod/MotroA03
2219,959086,journals/sigmod/KambhampatiK03


In [971]:
record

,idACM,idDBLP
0,277955,conf/sigmod/Epstein95
1,277955,conf/sigmod/Ellison95
2,276377,conf/sigmod/OlstonWACELSS98
3,276367,conf/sigmod/Aulakh98
4,304211,journals/vldb/MaratheS02
...,...,...
923,672978,conf/vldb/JonscherD94
924,950483,journals/vldb/RamamurthyDS03
925,672977,conf/vldb/KemperK94
926,672980,conf/vldb/Guting94


### Preparing the data for analysis

Precision, recall, and F1-measure can only be calculated if the dataframes are the same size. Therefore, need to shape the dataframes accordingly.

The approach chosen was to select a subsets from both datasets where the `idACM` is matching.

In [972]:
def get_perfect_subset(perfect_df, matched_df):
  # Create a new dataframe with matching ids
  ids = matched_df['idACM'].values
  perfect_subset = perfect_df[perfect_df['idACM'].isin(ids)]

  # Return the matchin ids
  return perfect_subset['idACM'].values

In [973]:
record_new = record.drop_duplicates(subset ="idACM", keep = False)

ids_perfect = get_perfect_subset(perfect, record_new)

record_new = record_new[record_new['idACM'].isin(ids_perfect)].sort_values(by=['idACM'])

perfect_record = perfect[perfect['idACM'].isin(ids_perfect)].sort_values(by=['idACM'])


print(record_new.shape)
print(perfect_record.shape)


(896, 2)
(896, 2)


In [981]:
ids_perfect = get_perfect_subset(perfect, fuzzy)

fuzzy_new = fuzzy[fuzzy['idACM'].isin(ids_perfect)].sort_values(by=['idACM'])

perfect_fuzzy = perfect[perfect['idACM'].isin(ids_perfect)].sort_values(by=['idACM'])

print(fuzzy_new.shape)
print(perfect_fuzzy.shape)


(2201, 2)
(2201, 2)


This results in perfect and matched dataframes in the same sizes

### Results

In [975]:
scores_r = precision_recall_fscore_support(perfect_record['idDBLP'].values, 
                                record_new['idDBLP'].values, 
                                average='weighted', zero_division=0)

In [976]:
scores_f = precision_recall_fscore_support(perfect_fuzzy['idDBLP'].values,
                                fuzzy_new['idDBLP'].values,
                                average='macro', zero_division=0)

Fuzzy matcher approach scores

In [977]:
print(f"Precision: {scores_f[0]}\nRecall: {scores_f[1]}\nF1 score: {scores_f[2]}")

Precision: 0.9887755102040816
Recall: 0.9909297052154195
F1 score: 0.9894482237339381


Python Record Linkage approach scores

In [978]:
print(f"Precision: {scores_r[0]}\nRecall: {scores_r[1]}\nF1 score: {scores_r[2]}")

Precision: 0.9933035714285714
Recall: 0.9955357142857143
F1 score: 0.994047619047619


### Analysis

The variables for analysis are precision, recall, and F1-measure as suggested by the coursework.



*   "Precision - Precision is the ratio of correctly predicted positive observations to the total predicted positive observations. High precision relates to the low false positive rate."
*   "Recall - Recall is the ratio of correctly predicted positive observations to the all observations in actual class." The higher recall is the more correctly predicted the data was.
*   "F1 score - F1 Score is the weighted average of Precision and Recall".

The scores for each measure is between 0 and 1





All of the scores are very close to 1, which indicates that the matches are very close to the perfectly matched data set. Therefore, the record matching with both approaches is a success.

Looking into the precision, recall and f1 scores, the best method looks to be python record linkage toolkit. However, it's important to note how much records were actually matched.

In [979]:
len(fuzzy_new)

2201

In [980]:
len(record_new)

896

Therefore, even though the record linkage toolkit matched the records more accuratelly, the fuzzy matcher matched more than two times more records. And considering that the scores were not that apart, the fuzzymatcher can be considered the better approach here.

### Extension of the research

To extend this research it would be interesting to look into:


*   how the different parameters of the methods influence the performace and effectiveness. Especially with record linkage toolkit full_blocking indexer.
